# ARTIGO 04/06 — Robustez (Game-theoretic + Execução)

- `research/ARTIGO 4 - Teoria dos Jogos aplicada ao Treinamento de Agentes DRL em Criptomoedas.md`
- `research/ARTIGO 6 - DRL em Criptomoedas sob Restrições de Execução - Custos, Impacto e Robustez.md`

Este notebook é um scaffold para comparar tratamentos do tipo:
- RL puro vs RL + projeção (restrições)
- treino robusto (domain randomization) vs baseline
- (futuro) self-play / max-min


In [ ]:
RUN_IDS = []
RUNS_DIR = "runs"


In [ ]:
from pathlib import Path
import sys

import pandas as pd

cwd = Path.cwd().resolve()
repo_root = cwd if (cwd / "crates").exists() else cwd.parent
notebooks_dir = repo_root / "notebooks"
sys.path.insert(0, str(notebooks_dir / "_lib"))

from runs import load_run, latest_run
from dataframes import read_trades_csv, read_equity_csv
from plots import plot_equity_curve

runs_dir = (repo_root / RUNS_DIR).resolve()
if not RUN_IDS:
    rd = latest_run(runs_dir)
    if rd is None:
        raise RuntimeError(f"no runs found under {runs_dir}")
    RUN_IDS = [rd.name]

loaded = [load_run(runs_dir / rid) for rid in RUN_IDS]
[(r.run_id, bool(r.summary)) for r in loaded]

## Summary


In [ ]:
rows = []
for r in loaded:
    s = r.summary or {}
    rows.append({
        "run_id": r.run_id,
        "trades": s.get("trades"),
        "net_profit": s.get("net_profit"),
        "sharpe": s.get("sharpe"),
        "max_drawdown": s.get("max_drawdown"),
    })
pd.DataFrame(rows)

## Equity curves


In [ ]:
for r in loaded:
    if r.equity_csv is None:
        continue
    eq = read_equity_csv(r.equity_csv)
    _, figures_dir = r.ensure_paper_dirs()
    out = figures_dir / f"equity_{r.run_id}.png"
    plot_equity_curve(eq, title=f"Equity — {r.run_id}", save_path=out)
    print(out)

## TODO (ARTIGO 04/06)

- Agrupar por tratamento (T1/T2/T3) e por cenário de fricção (custo/impacto/limites).
- Exportar tabelas de robustez: pior-caso (quantis/CVaR), variância por seed/janela.
- Quando `manifest.json` existir, filtrar automaticamente por treatment_id/cost_scenario.
